In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

### Custom files ###
dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

# Import
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_Matern52

Unable to determine R home: [WinError 2] The system cannot find the file specified
R[write to console]: Loading required package: mvtnorm



Loading constrained GP module from C:\Users\tln229\Downloads\Python\1. Building\Finals\3. Thermostat model\predict command value
Loading R wrapper...


R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: gmm

R[write to console]: Loading required package: sandwich

R[write to console]: 
Attaching package: 'TruncatedNormal'


R[write to console]: The following objects are masked from 'package:tmvtnorm':

    dtmvnorm, dtmvt, ptmvnorm, ptmvt, rtmvnorm, rtmvt


R[write to console]: The following objects are masked from 'package:mvtnorm':

    pmvnorm, pmvt




Running R from rpy2: R version 4.2.2 (2022-10-31 ucrt)


In [2]:
def get_diagnostics_df(x_test, y_test, mean, var):
    """ Put test data in a dataframe"""
    df = pd.DataFrame(x_test)
    df.columns = ['x' + str(i+1) for i in range(x_test.shape[1])]
    df['y_true'] = y_test
    df['y_mean'] = mean
    df['y_var'] = var
    return df

In [3]:
def constant_function(val):
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

In [4]:
# ---------- DATA ----------
def create_data(n_train, n_Xv):
    df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

    # TRAIN SET
    clg_sp       = np.array(df['clg_sp_current']).reshape(-1,1)[0: n_train]
    htg_sp       = np.array(df['htg_sp_current']).reshape(-1,1)[0: n_train]
    htg_clg_mode = 1*np.array(df['htg_clg_mode']).reshape(-1,1)[0: n_train]

    sp_k   = htg_sp*htg_clg_mode + clg_sp*(1-htg_clg_mode)
    Tz_k   = np.array(df['thermostat_room_temp']).reshape(-1,1)[0: n_train]
    qrh_k  = np.array(df['htg_valve_command']).reshape(-1,1)[0: n_train]
    qrh_k1 = np.array(df['htg_valve_command'])[0+1: n_train+1]

    x_train  = np.concatenate((sp_k, Tz_k, qrh_k), axis=1)
    y_train = qrh_k1

    # Xv
    x1v = np.linspace(68, 76, n_Xv).reshape(-1, 1)
    x2v = np.linspace(68, 75, n_Xv).reshape(-1, 1)
    x3v = np.linspace(0, 100, n_Xv).reshape(-1, 1)
    Xv  = np.concatenate((x1v, x2v, x3v), axis = 1)

    # TEST SET
    clg_sp       = np.array(df['clg_sp_current']).reshape(-1,1)[n_train: 1600]
    htg_sp       = np.array(df['htg_sp_current']).reshape(-1,1)[n_train: 1600]
    htg_clg_mode = 1*np.array(df['htg_clg_mode']).reshape(-1,1)[n_train: 1600]

    sp_k   = htg_sp*htg_clg_mode + clg_sp*(1-htg_clg_mode)
    Tz_k   = np.array(df['thermostat_room_temp']).reshape(-1,1)[n_train: 1600]
    qrh_k  = np.array(df['htg_valve_command']).reshape(-1,1)[n_train: 1600]
    qrh_k1 = np.array(df['htg_valve_command'])[n_train+1: 1600+1]
    msa_k1 = np.array(df['airflow_desired'])[n_train+1: 1600+1]

    x_test  = np.concatenate((sp_k, Tz_k, qrh_k), axis=1)
    y_test  = qrh_k1
    y2_test = msa_k1

    return x_train, y_train, Xv, x_test, y_test, y2_test

In [5]:
df_result = pd.DataFrame({'n_train':[], 'n_Xv':[], 'R2_qrh':[], 'R2_msa':[]})

for n_train in ([32, 64, 128]):
    for n_Xv in ([5, 10, 15]):
        # DATA
        x_train, y_train, Xv, x_test, y_test, y2_test = create_data(n_train=n_train, n_Xv=n_Xv)

        # MODEL
        ker = kernel_Matern52(variance = 2, lengthscale = [0.2]*x_train.shape[1])
        model = GPmodel(kernel = ker, likelihood = 1E-4, mean = 0)

        model.X_training = x_train
        model.Y_training = y_train

        model.optimize(include_constraint = False, fix_likelihood = False)

        # CONSTRAINTS
        model.constr_bounded    = Constraint(LB = constant_function(0), UB = constant_function(100))
        model.constr_bounded.Xv = Xv
        model.constr_likelihood = 1E-6

        # PREDICT qrh
        mean, _, _, _, _, _ = model.calc_posterior_constrained(x_test, num_samples = 5000, save_samples = 500)
        mean = np.array(mean).flatten()

        # PREDICT msa
        df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')
        qrh = np.array(df['htg_valve_command']).reshape(-1,1)[0: n_train]
        msa = np.array(df['airflow_desired']).reshape(-1,1)[0: n_train]

        A = np.concatenate((qrh, np.ones(msa.shape)), axis=1)
        b = np.copy(msa)
        p = np.linalg.lstsq(A, b, rcond=None)[0]
        pred_msa = mean*p[0] + p[1]

        print(n_train, n_Xv, r2_score(y_test, mean), r2_score(y2_test, pred_msa))

        df_result.loc[len(df_result)] = [n_train, n_Xv, r2_score(y_test, mean), r2_score(y2_test, pred_msa)]

..Running optimization for unconstrained GP ... DONE - Total time: 0.069 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.075 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.000 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.010 seconds
..sampling 5000 times from truncated constraint distribution C~|C, Y DONE - time: 0.017 seconds
..sampling 5000 times from constrained GP f*|C, Y DONE - time: 4.146 seconds
..computing statistics from samples DONE - time: 0.203 seconds
 DONE - Total time: 4.452 seconds
32 5 0.9050629528479801 0.9048402863484547
..Running optimization for unconstrained GP ... DONE - Total time: 0.066 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - de

In [6]:
print(df_result)

   n_train  n_Xv    R2_qrh    R2_msa
0     32.0   5.0  0.905063  0.904840
1     32.0  10.0  0.907064  0.906838
2     32.0  15.0  0.906168  0.905943
3     64.0   5.0  0.974525  0.974035
4     64.0  10.0  0.974446  0.973957
5     64.0  15.0  0.974495  0.974006
6    128.0   5.0  0.973812  0.973246
7    128.0  10.0  0.973830  0.973263
8    128.0  15.0  0.973822  0.973255
